In [0]:
from pyspark.sql import functions as F
from pyspark.sql.types import LongType, StringType, BooleanType, TimestampType, IntegerType, ShortType
from delta.tables import DeltaTable
from datetime import datetime
from pyspark.sql.functions import col, regexp_replace, sha2, when, lit, uuid

CATALOGO_ORIGEM = "v_credit"
SCHEMA_ORIGEM = "bronze"
TABELA_ORIGEM = "clientes"

CATALOGO_DESTINO = "v_credit"
SCHEMA_DESTINO = "silver"
TABELA_DESTINO = "tb_cliente"
TABELA_INVALIDOS_DESTINO = "tb_cliente_invalidos"

nome_tabela_origem = f"{CATALOGO_ORIGEM}.{SCHEMA_ORIGEM}.{TABELA_ORIGEM}"
nome_tabela_destino = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_DESTINO}"
nome_tabela_invalidos = f"{CATALOGO_DESTINO}.{SCHEMA_DESTINO}.{TABELA_INVALIDOS_DESTINO}"

In [0]:
cliente_bronze_df = spark.read.table(nome_tabela_origem)

df_transformado = (
    cliente_bronze_df
    .select(
        F.substring(F.sha2(F.col("id_cliente").cast(StringType()), 256), 1, 36).alias("cd_cliente"),
        F.sha2(F.col("id_cliente").cast(StringType()), 256).alias("nu_cpf"),
        F.trim(F.col("nome")).alias("nm_cliente"),
        F.regexp_replace(F.col("email"), r'\+[A-Za-z0-9._%+-]+@', "@").alias("ds_email"),
        F.trim(F.col("regiao")).alias("nm_regiao"),
        F.col("idade").cast(ShortType()).alias("nu_idade"),
        F.col("ingestion_timestamp").alias("dt_ingestion"), 
        F.col("origem").alias("dc_origem")
    )
    .dropDuplicates(["nu_cpf"])
)

In [0]:
df_validacao = (
    df_transformado
    .withColumn("flag_email_valido", F.col("ds_email").rlike(EMAIL_REGEX))
    .withColumn("flag_maior_idade", F.col("nu_idade") >= 18)
    .withColumn("flag_cpf_valido", F.col("nu_cpf").isNotNull())
    .withColumn("flag_qualidade",
        F.when(
            F.col("flag_email_valido") & 
            F.col("flag_maior_idade") & 
            F.col("flag_cpf_valido"),
            F.lit("OK")
        ).otherwise(F.lit("ERRO"))
    )
)

df_validos = df_validacao.filter(F.col("flag_qualidade") == "OK")
df_invalidos = df_validacao.filter(F.col("flag_qualidade") == "ERRO")

df_silver = df_validos.select(
    "cd_cliente",
    "nu_cpf",
    "nm_cliente",
    "ds_email",
    "nm_regiao",
    "nu_idade",
    "dt_ingestion",
    "dc_origem"
)

In [0]:
delta_table = DeltaTable.forName(spark, nome_tabela_destino)

delta_table.alias("destino").merge(
    df_silver.alias("origem"),
    "destino.nu_cpf = origem.nu_cpf"
).whenMatchedUpdateAll(
).whenNotMatchedInsertAll(
).execute()

print(f"✅ Tabela {nome_tabela_destino} atualizada com sucesso!")
print(f"Total de registros válidos processados: {df_silver.count()}")

df_invalidos.write.format("delta").mode("overwrite").saveAsTable(nome_tabela_invalidos)

print(f"✅ Tabela {nome_tabela_invalidos} atualizada para auditoria!")
print(f"Total de registros inválidos: {df_invalidos.count()}")